In [1]:
#import scanpy as sc
#import scipy as sci
#import numpy as np
#import anndata
#import time
#from sklearn.decomposition import PCA
#import fast_histogram
#import logging
#logging.basicConfig(level=logging.INFO)
#import pandas as pd

#import numba

prep.py component \
Read input file and slice into manageable sizes

In [2]:
#from MICA.lib import utils

In [3]:
#from IPython import parallel
import ipyparallel as ipp

In [4]:
#We need to launch an ipython parallel cluster
#run this on hpc node to launch a cluster with mpi engines
#ipcluster start --engines=MPIEngineSetLauncher --log-level DEBUG --n=3 &
#create a parallel client so that we can use %%px cell magic
#rc = parallel.Client()
rc = ipp.Client()
dview = rc[:]
rc.ids

[0, 1, 2]

In [19]:
#load all necessary libraries onto each rank
from scipy.sparse import csr_matrix
from mpi4py import MPI
import sys
import numba
import pandas as pd
import scanpy as sc
import scipy as sci
import numpy as np
import anndata
import time
from sklearn.decomposition import PCA
import fast_histogram
import logging
logging.basicConfig(level=logging.INFO)
from MICA.lib import utils

In [20]:
%%px
#from mpi4py import MPI
#import sys

comm = MPI.COMM_WORLD
#rank = comm.Get_rank()
size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()
sys.stdout.write(
    "Hello, World! I am process %d of %d on %s.\n" 
    % (rank, size, name))

[stdout:0] Hello, World! I am process 0 of 3 on noderome166.
[stdout:1] Hello, World! I am process 1 of 3 on noderome166.
[stdout:2] Hello, World! I am process 2 of 3 on noderome166.


In [7]:
#%%file hellompi.py
#"""
#Parallel Hello World
#"""
#from mpi4py import MPI
#import sys
#MPI_Init(NULL, NULL);
#MPI_Comm_size(MPI_COMM_WORLD, &world_size);
#size = MPI.COMM_WORLD.Get_size()
#rank = MPI.COMM_WORLD.Get_rank()
#name = MPI.Get_processor_name()
#sys.stdout.write(
#    "Hello, World! I am process %d of %d on %s.\n" 
#    % (rank, size, name))

In [8]:
#!mpiexec -n 3 python3 hellompi.py

In [9]:
#def read_anndata_file(in_file):
#    """ Reads text file and stores data in a temporary HDF5-format file.
#
#    Args:
#        in_file_name  (str): path to input text file
#        out_file_name (str): user-defined name for output file
#    """
#    if in_file.endswith('.h5ad'):
#        adata = anndata.read_h5ad(in_file)
#    #write dataframe to hdf
#    #adata.to_hdf(out_file_name + ".h5.tmp", "slice_0")
#    return adata

In [21]:
#this is a defined elsewhere as a standalone python executable
#we are duplicating here to experiment with format for distrubuted computing
def prep_dist(input_file, out_name, slice_unit):
    """ Preprocess input file to create sliced matrices.

    Reads file into HDF5-format, adds parameters, slices data in file, generates several files with
    different combinations of slices.

    Args:
        input_file (str): path to input text-file
        out_name   (str): common rootname of generated output files
        slice_unit (int): size of each slice of cell data in input text-file
    """
    logging.basicConfig(level=logging.INFO)
    
    #Read in whole file stored in anndata csr format
    #adf=utils.read_anndata_file(in_file)
    adf=utils.read_anndata_file(input_file)


    #anndata object can be split easily by subsetting

    #patch_file
    """ Prepares the HDF5 file for slicing. Completes the "temporary" HDF5-format file.
    Reads input file into several data frames. Indexes attributes as row, col and slice.
    Indexes columns and rows of data. Converts all data frames into an output HDF5 file
    with separate keys for each piece of data.
    Args:
        df_file       (str): path to HDF5-format file
        out_file_name (str): path to complete HDF5-format file
    """
    #patch adds parameters to anndata object
    #ceb I'm not sure what value the patch function adds
    
    # prepare h5 files (whole)
    #h5_tmp = out_name + ".h5ad.tmp"
    #utils.patch_anndata_csr_file(h5_tmp, out_name)

    #adata_sub = utils.patch_anndata_csr_file(adf, out_name)

    #os.remove(h5_tmp)
    
    #def patch_file(df_file,  out_file_name):
    #df = pd.HDFStore(df_file)["slice_0"]
    #df.to_hdf(out_file_name + ".whole.h5", "slice_0")
    #pd.DataFrame(data=np.array(df.shape + (1,)), index=["row", "col", "slice"]).to_hdf(
    #    out_file_name + ".whole.h5", "attr"
    #)
    #pd.DataFrame(data=df.columns, index=df.columns).to_hdf(
    #    out_file_name + ".whole.h5", "cols"
    #)
    #pd.DataFrame(data=df.index, index=df.index).to_hdf(
    #    out_file_name + ".whole.h5", "rows"
    #)    
    
    
    #Slice_file
    #def slice_file(df_file,  out_file_name, slice_size="1000"):
    """ Slices the HDF5 file.
    Determines the number of slices for the data, based on slice_size.
    Calculates start and end indices for slicing, creating a new dataframe
    based on those indices and appends them to the sliced output file using
    a unique-identifier in the format 'slice_00x' as key.
    Args:
        df_file       (str): path to HDF5-format file
        out_file_name (str): path to sliced output HDF5 file
        slice_size    (str): number of items in each slice
    """
    
    #h5_whole = out_name + ".whole.h5ad"
    slice_size = int(slice_unit)
    
    #df = pd.HDFStore(df_file)["slice_0"]
    
    #compute number of slices needed to break dataset in to slice_size batches
    b = int(np.ceil(float(adf.shape[0]) / float(slice_size)))
    #determine how many digits are in b so we can pad spaces for the string output
    digit = int(np.floor(np.log10(b)) + 1)
    #loop over slice numbers
    for i in range(b):
        #slice name is equal to batch index
        slice_name = str(i).zfill(digit)
        #compute batch row indices
        start = i * slice_size
        end = np.min([(i + 1) * slice_size, adf.shape[0]])
        #copy slice to array of slices
        adf_sub=adf[start:end,:]
        #write to file so we don't have to keep each slice in memory
        output_file_name = out_name + ".slice_" + slice_name +".h5ad"
        print("output_file_name: ",output_file_name)
        adf_sub.write(output_file_name)
        
 
    
    #return nrows and nslices
    return adf.shape[0], adf.shape[1], b
    
    #ceb this function creates pairwise block comparison files for each b(b-1)/2 block comparisons to be done.
    #calc_mi function is called to run independently on each of these comparison files.
    #We can replace this if we use mpi to spawn ranks and then have each rank perform one or more of these block comparisons.
    # The mpi rank will read the two files that it needs, so we can reduce the number of files generated
    # We can repurpose the following loop to direct the matrix file to the mpi ranks
    

  
    #in_.close()    
    #logging.info('MICA-prep step completed successfully.')

In [22]:
%%px
#ceb create csr version of numba_histogram2d, also compute_bin with knowledge that minx will always be zero

@numba.jit(nopython=True)
def numba_nan_fill(x):
    shape = x.shape
    x = x.ravel()
    x[np.isnan(x)] = 0.0
    x = x.reshape(shape)
    return x

@numba.jit(nopython=True)
def numba_inf_fill(x):
    shape = x.shape
    x = x.ravel()
    x[np.isinf(x)] = 0.0
    x = x.reshape(shape)
    return x

@numba.jit(nopython=True, fastmath=True)
def compute_bin_upperbound(x, max, num_bins):
    """ Compute bin index for a give number.
        Assume that min is always zero
    """
    # special case to mirror NumPy behavior for last bin
    if x == max:
        return num_bins - 1 # a_max always in last bin

    bin = int(num_bins * x / max)

    if bin >= num_bins:
        return None
    else:
        return bin

@numba.jit(nopython=True, fastmath=True)
def numba_histogram2d_csr(arr1, cols1, arr2, cols2, ncols, num_bins):
    """ Compute the bi-dimensional histogram of two data samples.
    Args:
        arr1 (array_like, shape (N,)): An array containing the x coordinates of the points to be histogrammed.
        arr2 (array_like, shape (N,)): An array containing the y coordinates of the points to be histogrammed.
        num_bins (int): int
    Return:
        hist (2D ndarray)
    """
    #for csr arrays we have to compute zero bins ahead of time 
        
    bin_indices1 = np.zeros((ncols,), dtype=np.int16)
    max1 = arr1.max()
    #note that bin_indices has same size/indices as full array x and y
    for i, x in enumerate(arr1.flat):
        #assume zero min
        bin_indices1[cols1[i]] = compute_bin_upperbound(x, max1, num_bins)
        #bin_indices1[cols1[i]] = compute_bin(x, 0, max1, num_bins)

    bin_indices2 = np.zeros((ncols,), dtype=np.int16)
    max2 = arr2.max()
    for i, y in enumerate(arr2.flat):
        #assume zero min
        bin_indices2[cols2[i]] = compute_bin_upperbound(y, max2, num_bins)
        #bin_indices2[cols2[i]] = compute_bin(y, 0, max2, num_bins)

    hist = np.zeros((num_bins, num_bins), dtype=np.int16)
    for i, b in enumerate(bin_indices1):
        hist[b, bin_indices2[i]] += 1
        
    return hist

In [23]:
%%px
import numba
@numba.jit(nopython=True, fastmath=True)
def numba_calc_mi_dis_csr(arr1, cols1, arr2, cols2, bins, m):
    """ Calculates a mutual information distance D(X, Y) = H(X, Y) - I(X, Y) using bin-based method

    It takes gene expression data from single cells, and compares them using standard calculation for
    mutual information and joint entropy. It builds a 2d histogram, which is used to calculate P(arr1, arr2).

    Args:
        arr1 (float) nparray of csr: gene expression data for cell 1
        cols1 (int): column indices for csr arr1
        arr2 (float) nparray of csr: gene expression data for cell 2
        cols2 (int): column indices for csr arr
        bins           (int): number of bins
        m              (int): number of genes
    Returns:
        a float between 0 and 1
    """
    hist = numba_histogram2d_csr(arr1, cols1, arr2, cols2, m, bins)
    
    sm = np.sum(hist, axis=1)
    tm = np.sum(hist, axis=0)
    sm = sm / float(sm.sum())
    tm = tm / float(tm.sum())

    sm_tm = np.zeros((bins, bins), dtype=np.float32)
    for i, s in enumerate(sm):
        for j, t in enumerate(tm):
            sm_tm[i, j] = s * t

    fq = hist / float(m)
    div = np.true_divide(fq, sm_tm)
    numba_nan_fill(div)
    ent = np.log(div)
    numba_inf_fill(ent)
    agg = np.multiply(fq, ent)
    joint_ent = -np.multiply(fq, numba_inf_fill(np.log(fq))).sum()
    return joint_ent - agg.sum()

In [24]:
%%px
#numba compilation cannot interpret the creation of a 2d array inside of this function so we pass in and out SM_block instead of returning it
import numba
@numba.jit(nopython=True, fastmath=True)
def process_matrices(Arows,Amat_data,Amat_indptr,Amat_indices,
                     Brows,Bmat_data,Bmat_indptr,Bmat_indices,
                     num_bins,num_genes,
                     SM_block, symmetric=False
                    ):
    #(mat1.n_obs,mat1.X.data,mat1.X.indptr,mat1.X.indices, mat2.n_obs,mat2.X.data,mat2.X.indptr,mat2.X.indices,SM,num_bins,mat2.n_vars,symmetric)

    #Arows = mat1.n_obs
    #Brows = mat2.n_obs
    #num_genes = mat1.n_vars
    
    #SM_block = np.ndarray(shape=(Arows,Brows))#, dtype=float, order='F')
    if 0 :
        for i in range(Arows):
            Arowstart = Amat_indptr[i]
            Arowend   = Amat_indptr[i+1]
            Arow_cols = Amat_indices[Arowstart:Arowend]
            Arow_data = Amat_data[Arowstart:Arowend]
            for j in range(i,Brows):
                Browstart = Bmat_indptr[j]
                Browend   = Bmat_indptr[j+1]
                Brow_cols = Bmat_indices[Browstart:Browend]
                Brow_data = Bmat_data[Browstart:Browend]               

                SM_block[i,j] = numba_calc_mi_dis_csr(Arow_data, Arow_cols, Brow_data, Brow_cols, num_bins, num_genes)        
    else :
        
        for i in range(Arows):
            Arowstart = Amat_indptr[i]
            Arowend   = Amat_indptr[i+1]
            Arow_cols = Amat_indices[Arowstart:Arowend]
            Arow_data = Amat_data[Arowstart:Arowend]
            Bstart=i*int(symmetric)
            for j in range(Bstart,Brows):
                Browstart = Bmat_indptr[j]
                Browend   = Bmat_indptr[j+1]
                Brow_cols = Bmat_indices[Browstart:Browend]
                Brow_data = Bmat_data[Browstart:Browend]               

                SM_block[i,j] = numba_calc_mi_dis_csr(Arow_data, Arow_cols, Brow_data, Brow_cols, num_bins, num_genes)
            
            

    return #SM_block


In [74]:
%%px
from mpi4py import MPI
from scipy.sparse import csr_matrix
#import sys
import pandas as pd

#create a 2d list to hold blocks of similarity matrix
#this should be a global var
#SM = [[None for j in range(b)] for i in range(b)] 

def calc_distance_metric_distributed(in_file_path, project_name, nrows, ncols, nslices, SM):
    
    """ Prepares the already sliced input file for further calculation in MICA.
    Enters pairs of slices (matrices) into temporary HDF5-format files. It enters them
    individually, using their unique key. It also enters the parameter data for every single 
    pair into the key "params", which consists of: [key1, key2, num_bins, num_genes,
    pair_index, project_name, num_slices]
    Args:
        in_file      (str): path to sliced HDF5-format file
        project_name (str): project name used to generate path for final outputs
        nrows : number of rows in global matrix
        ncols : number of vars in global matrix
    """

    
    #create a 2d list to hold blocks of similarity matrix
    #this should be a global var
    #SM = [[None for j in range(b)] for i in range(b)] 
    
    
    #input file is full input that has been segmented into b blocks of rows
    
    #nranks would ideally be equal to  b(b+1)/2
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    myrank = comm.Get_rank()
    name = MPI.Get_processor_name()
    #sys.stdout.write("Hello, World! I am process %d of %d on %s.\n" % (myrank, size, name))
    
    #in_ = pd.HDFStore(in_file, "r")  # slice.h5
    #b = int(np.ceil(float(nrows) / float(slice_size)))
    digit = int(np.floor(np.log10(nslices)) + 1)

    
    num_bins = int(np.floor(nrows ** (1 / 3.0)))  # number of bins
    #print("bins= ",bins)
    #b = in_["attr"].loc["slice", 0]  # number of sliced matrix
    b = nslices
    m = ncols  # number of genes

    n_block_comparisons = int((b * (b + 1)) / 2)  # total number of calc jobs execute
    n_jobs_per_rank = n_block_comparisons/size
    if (myrank == 0): print("block comparsons = %d. jobs per rank = %d\n" % (n_block_comparisons, n_jobs_per_rank))

    #build list of slices that current mpi rank will process
    myslices=[]
    for i in range(b):
        for j in range(i,b): # j in range [i,b]
            idx = int(i * b + j - (i * (i + 1)) / 2)
            targetrank = idx//n_jobs_per_rank
            if (targetrank == myrank): myslices.append((i,j))            


    #only do work assigned to your rank
    for index, tuple in enumerate(myslices):
        print("tuple=",tuple)
        i=tuple[0] #block row
        j=tuple[1] #block col      
        
        #get 1st slice file
        #digit = int(np.floor(np.log10(i+1)) + 1)
        #digit = int(np.floor(np.log10(b)) + 1)

        slice_name = str(i).zfill(digit)
        ##ceb we only want to read this once per i,j combination
        input_file = in_file_path + project_name + ".slice_" + slice_name +".h5ad"
        print("infile seg1: ",input_file)
        mat1 = utils.read_anndata_file(input_file)
    

        #get 2nd slice file
        #digit = int(np.floor(np.log10(j+1)) + 1)
        slice_name = str(j).zfill(digit)
        input_file = in_file_path + project_name + ".slice_" + slice_name +".h5ad"
        print("infile seg2: ",input_file)
        mat2 = utils.read_anndata_file(input_file)    

        #check to see if block comparison will result in a symmetric SM matrix
        # so that we can reduce the number of computations in half
        symmetric=False
        if i==j: symmetric=True
        #print("",symmetric)    
        
        print("rank: ",myrank," comparison between segs:",i," x ",j," symmetric=",symmetric)
            
        #compute distance metrics between blocks
        
        Arows = mat1.n_obs
        Brows = mat2.n_obs
        num_genes = mat1.n_vars #we will assume Acols==Bcols==num_genes
        
        
        #df = pd.DataFrame(data=0, index=mat1.obs.index, columns=mat2.obs.index) 
        #need SM for each block pair
        #SM[i][j] = np.ndarray(shape=(Arows,Brows), dtype=float, order='C')
        
        SM[i][j] = np.zeros(shape=(Arows,Brows), dtype = float, order = 'C')
        
        ## store in csr matrix
        #SM[i][j] = csr_matrix((Arows, Brows), dtype=np.float32)
        

        #This numba function cannot create a numpy array internally so we return SM[i,j] as a variable
        process_matrices(mat1.n_obs,mat1.X.data,mat1.X.indptr,mat1.X.indices, 
                         mat2.n_obs,mat2.X.data,mat2.X.indptr,mat2.X.indices,
                         num_bins, num_genes, SM[i][j],
                         symmetric #if i==j we can eliminate half of computations
                        )

        #convert to csr
        #SM[i][j] = csr_matrix(data=SM[i][j],(row=mat1.n_obs,col=mat2.n_obs),shape=(Arows, Brows))
        #SM[i][j] = csr_matrix(SM[i][j],(Arows,Brows),shape=(Arows, 1000))
        SM[i][j]=csr_matrix(SM[i][j])

    return

In [72]:
import os
os.getcwd()

'/research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/MICA/analysis'

In [29]:
cwd = '/research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/'

In [66]:

data_file_path = cwd+'test_data/inputs/10x/PBMC/3k/pre-processed/'
input_file_name = data_file_path + 'pbmc3k_preprocessed.h5ad'
project_name = 'pbmc3k'

output_file_name = data_file_path+project_name
slice_size = 200

nrows, ncols, nslices = prep_dist(input_file_name, output_file_name, slice_size)

# cant remember what this does?
# I think it distributes these variables to all ranks
dview.push(dict(nrows=nrows, ncols=ncols, nslices=nslices))

output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_00.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_01.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_02.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_03.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_04.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_05.h5ad
output_file_name:  /research/rgs01/home/

<AsyncResult: _push>

In [67]:
print("global nrows, ncols, slices: ",nrows, ncols, nslices)

global nrows, ncols, slices:  2496 10499 13


Read in anndata preprocessed files (in distributed mode, by node number)

In [75]:
%%px

#create a 2d list to hold blocks of similarity matrix
#this should be stored in a distributed scalapack matrix
b=nslices
SM = [[None for j in range(b)] for i in range(b)] 

start = time.time()
calc_distance_metric_distributed(data_file_path, project_name, nrows, ncols, nslices, SM)
end = time.time()
print("Elapsed = %s" % (end - start))


[stdout:0] 
block comparsons = 91. jobs per rank = 30

tuple= (0, 0)
infile seg1:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_00.h5ad
infile seg2:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_00.h5ad
rank:  0  comparison between segs: 0  x  0  symmetric= True
tuple= (0, 1)
infile seg1:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_00.h5ad
infile seg2:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_01.h5ad
rank:  0  comparison between segs: 0  x  1  symmetric= False
tuple= (0, 2)
infile seg1:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_00.h5ad
infile 

In [ ]:
%%px 
from scipy.sparse import csr_matrix
if comm.Get_rank() == 0:
    #print(SM[0][0].shape)
    #B=csr_matrix(SM[0][0]) 
    print(SM[0][0].getnnz)

In [41]:
%%px 
from scipy.sparse import csr_matrix
if comm.Get_rank() == 0:
    SM_csr=csr_matrix(SM[0][0],(1000,1000),shape=(1000, 1000))


CompositeError: one or more exceptions from call to method: execute
[0:execute]: TypeError: __init__() got multiple values for argument 'shape'

In [37]:
%%px
#show that diagonal blocks are symmetric
if comm.Get_rank() == 0:
    i=1
    j=100
    print((SM[0][0])[i,j])
    print((SM[0][0])[j,i])

[stdout:0] 
0.6541460427482868
0.0


In [ ]:
#adata_sub1=adata[0:1000,:]
#adata_sub2=adata[1000:2000,:]
#adata_sub3=adata[2000:,:]

#print(adata_sub1.shape)
#print(adata_sub2.shape)
#print(adata_sub3.shape)

In [ ]:
#We can split the anndata object by rows in the following way:
#print(adata.X.indptr)
#print(adata_sub1.X.indptr)
#print(adata_sub2.X.indptr)
#print(adata_sub3.X.indptr)

In [ ]:
#Convert to dataframe for comparison
aframe = adata.to_df()
bframe = bdata.to_df()
aframe.shape

In [ ]:
#print(frame.iloc[2000,:])
#(frame == 0).astype(int).sum(axis=1)
#print(frame[frame == 0].count(axis=1)/len(frame.columns))

In [ ]:
#frame.to_csv('../../test_data/outputs/kgraph/pmbc3k.csv')

In [ ]:
def calc_distance_mat_dist(mat1, mat2, paras, method):
    """ Calculates a distance metric in between two matrices (slices)

    Calculates a distance metric using the preferred method of comparison. Iterates over each cell's
    gene expression data and populates a new matrix with rows and columns as cells from the input
    matrices. The resulting matrix is then converted to an HDF5-format file.

    Args:
        mat1  (anndata dataframe): a sliced part of the original matrix, with some fraction of the
                                  total cells as rows from original file and all gene expression
                                  attributes as columns
        mat2  (anndata dataframe): similar to mat1
        paras (anndata dataframe): a dataframe that holds an array of parameters from the whole dataset
        method             (str): the method to be used for the distance calculation (
                                        mutual information: "mi")
    """

    bins = int(paras.loc["num_bins", 0])
    m = int(paras.loc["n_genes", 0])
    key = paras.loc["MI_indx", 0]

    project_name = paras.loc["project_name", 0]
    out_file_name = project_name + "_" + key + ".h5"
    print(out_file_name)

    if method == "mi":

        df = pd.DataFrame(data=0, index=mat1.index, columns=mat2.index) 
        start = time.time()
        
        for c in mat2.index:
            df.loc[mat1.index, c] = mat1.apply(
                calc_mi, axis=1, args=(mat2.loc[c, :], bins, m)
            )
            
        end = time.time()

    else:
        sys.exit("Distance Metrics not supported!\n")

        
    df.to_hdf(out_file_name, str(key))  # key: MI_indx
    paras.to_hdf(out_file_name, "params")


In [ ]:
    #tmp hardcode input file
    inputfileA = '../../test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad'
    inputfileB = '../../test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad'

    #For given rank id, read in A and B matrix slices 
    matA = anndata.read_h5ad(inputfileA)
    matB = anndata.read_h5ad(inputfileB)
    metrics = metric.lower()
    params = matA["params"]

In [ ]:
#reproducing standalone calc_scatter function here to experiment with distributed processing
def calc_scatter(input_file, metric='mi'):
    """ Calls calc_distance_mat utility function and calculates a metric in between cells that is chosen by the user

    Args:
        input_file (str): path to input HDF5-format file
        metric     (str): metric for calculation (mutual info, euclidean dist, pearson or spearman correlations
    """

    #mat = pd.HDFStore(input_file)
    #metrics = metric.lower()
    #params = mat["params"]
    #mat1 = mat[params.loc["key1", 0]]
    #mat2 = mat[params.loc["key2", 0]]
    #mat.close()
    
    #ceb set up an MPI rank list and coordinator to read in appropriate matrix slice files, 
    # compute distance matrix blocks and store results in distributed distance matrix

    #tmp hardcode input file
    inputfileA = '../../test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad'
    inputfileB = '../../test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad'

    #For given rank id, read in A and B matrix slices 
    matA = anndata.read_h5ad(inputfileA)
    matB = anndata.read_h5ad(inputfileB)
    metrics = metric.lower()
    params = matA["params"]

    #utils.calc_distance_mat(matA, matB, params, method=metrics)
    calc_distance_mat_distributed(matA, matB, params, method=metrics)

In [ ]:
inputfile = '../../test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad'
calc_scatter(inputfile,'mi')

In [ ]:
#%memit
start = time.time()
embedding = PCA(n_components=100)
frame_dr = embedding.fit_transform(frame)
frame_dr.shape
end = time.time()
runtime = end - start
msg = "The runtime for PCA took {} seconds to complete".format(runtime)
logging.info(msg)



In [ ]:
%load_ext memory_profiler

In [ ]:
def calc_mi_f(arr1, arr2, bins, m):
    """ Calculates mutual information in between two cells, considering their gene expression levels

    This function is called by calc_distance_mat. It takes gene expression data from single cells,
    and compares them using standard calculation for mutual information. It builds a 2d histogram,
    which is used to calculate P(arr1, arr2)

    Args:
        arr1 (pandas series): gene expression data for a given cell in matrix_1
        arr2 (pandas series):
        bins           (int):
        m              (int):
    """
    fq = fast_histogram.histogram2d(arr1, arr2, range=[[arr1.min(), arr1.max()+1e-9], [arr2.min(), arr2.max()+1e-9]],
                                    bins=(bins, bins)) / float(m)
    sm = np.sum(fq * float(m), axis=1)
    tm = np.sum(fq * float(m), axis=0)
    sm = np.asmatrix(sm / float(sm.sum()))
    tm = np.asmatrix(tm / float(tm.sum()))
    sm_tm = np.matmul(np.transpose(sm), tm)
    div = np.divide(fq, sm_tm, where=sm_tm != 0, out=np.zeros_like(fq))
    ent = np.log(div, where=div != 0, out=np.zeros_like(div))
    agg = np.multiply(fq, ent, out=np.zeros_like(fq), where=fq != 0)
    return agg.sum()

In [ ]:
num_bins = int((frame_dr.shape[0]) ** (1 / 3.0))
num_genes = frame_dr.shape[1]

In [ ]:
%timeit calc_mi_f(frame_dr[0], frame_dr[1], num_bins, num_genes)

In [ ]:
arr = frame_dr[0]
fast_histogram.histogram1d(arr, bins=num_bins, range=[arr.min(), arr.max()+1e-9]) / num_genes


In [ ]:
num_cells = frame_dr.shape[0]
marginals = np.empty((num_cells, num_bins))
for index, cell in enumerate(frame_dr):
    ht1d = fast_histogram.histogram1d(cell, bins=num_bins, range=[cell.min(), cell.max()+1e-9]) / num_genes
    marginals[index] = ht1d
print(marginals[0])
print(marginals[1])
np.transpose(np.asmatrix(marginals[0]))

In [ ]:
def calc_mi_f2(arr1, arr2, marginals, index1, index2, bins, m):
    """ Calculates mutual information in between two cells, considering their gene expression levels

    This function is called by calc_distance_mat. It takes gene expression data from single cells,
    and compares them using standard calculation for mutual information. It builds a 2d histogram,
    which is used to calculate P(arr1, arr2)

    Args:
        arr1 (pandas series): gene expression data for a given cell in matrix_1
        arr2 (pandas series):
        bins           (int):
        m              (int):
    """
    fq = fast_histogram.histogram2d(arr1, arr2, range=[[arr1.min(), arr1.max()+1e-9], [arr2.min(), arr2.max()+1e-9]],
                                    bins=(bins, bins)) / float(m)
    sm_tm = np.matmul(np.transpose(np.asmatrix(marginals[index1])), np.asmatrix(marginals[index2]))
    div = np.divide(fq, sm_tm, where=sm_tm != 0, out=np.zeros_like(fq))
    ent = np.log(div, where=div != 0, out=np.zeros_like(div))
    agg = np.multiply(fq, ent, out=np.zeros_like(fq), where=fq != 0)
    return agg.sum()


In [ ]:
#%timeit
calc_mi_f2(frame_dr[0], frame_dr[1], marginals, 0, 1, num_bins, num_genes)

In [ ]:
def calc_marginals(frame_dr, num_bins, num_genes):
    num_cells = frame_dr.shape[0]
    marginals = np.empty((num_cells, num_bins))
    for index, cell in enumerate(frame_dr):
        ht1d = fast_histogram.histogram1d(cell, bins=num_bins, range=[cell.min(), cell.max() + 1e-9]) / num_genes
        marginals[index] = ht1d
    np.transpose(np.asmatrix(marginals[0]))
    return marginals

In [ ]:
def calc_norm_mi_marginal(arr1, arr2, marginals, index1, index2, bins, m):
    """ Calculates a normalized mutual information distance in between two cells

    It takes gene expression data from single cells, and compares them using standard calculation for
    mutual information. It builds a 2d histogram, which is used to calculate P(arr1, arr2)

    Args:
        arr1 (pandas series): gene expression data for a given cell in matrix_1
        arr2 (pandas series):
        bins           (int):
        m              (int):
    """
    fq = fast_histogram.histogram2d(arr1, arr2, range=[[arr1.min(), arr1.max()+1e-9], [arr2.min(), arr2.max()+1e-9]],
                                    bins=(bins, bins)) / float(m)
    sm_tm = np.matmul(np.transpose(np.asmatrix(marginals[index1])), np.asmatrix(marginals[index2]))
    div = np.divide(fq, sm_tm, where=sm_tm != 0, out=np.zeros_like(fq))
    ent = np.log(div, where=div != 0, out=np.zeros_like(div))
    agg = np.multiply(fq, ent, out=np.zeros_like(fq), where=fq != 0)
    joint_ent = -np.multiply(fq, np.log(fq, where=fq != 0, out=np.zeros_like(fq)),
                             out=np.zeros_like(fq), where=fq != 0).sum()
    return (joint_ent - agg.sum()) / joint_ent

In [ ]:
# %timeit
calc_norm_mi_marginal(frame_dr[0], frame_dr[1], marginals, 0, 1, num_bins, num_genes)

In [ ]:
calc_norm_mi_marginal(frame_dr[0], frame_dr[2], marginals, 0, 2, num_bins, num_genes)



In [ ]:
def calc_norm_mi(arr1, arr2, bins, m):
    """ Calculates a normalized mutual information distance in between two cells

    It takes gene expression data from single cells, and compares them using standard calculation for
    mutual information. It builds a 2d histogram, which is used to calculate P(arr1, arr2)

    Args:
        arr1 (pandas series): gene expression data for a given cell in matrix_1
        arr2 (pandas series):
        bins           (int):
        m              (int):
    """
    fq = fast_histogram.histogram2d(arr1, arr2, range=[[arr1.min(), arr1.max()+1e-9], [arr2.min(), arr2.max()+1e-9]],
                                    bins=(bins, bins)) / float(m)
    sm = np.sum(fq * float(m), axis=1)
    tm = np.sum(fq * float(m), axis=0)
    sm = np.asmatrix(sm / float(sm.sum()))
    tm = np.asmatrix(tm / float(tm.sum()))
    sm_tm = np.matmul(np.transpose(sm), tm)
    div = np.divide(fq, sm_tm, where=sm_tm != 0, out=np.zeros_like(fq))
    ent = np.log(div, where=div != 0, out=np.zeros_like(div))
    agg = np.multiply(fq, ent, out=np.zeros_like(fq), where=fq != 0)
    joint_ent = -np.multiply(fq, np.log(fq, where=fq != 0, out=np.zeros_like(fq)),
                             out=np.zeros_like(fq), where=fq != 0).sum()
    return (joint_ent - agg.sum()) / joint_ent

In [ ]:
%timeit calc_norm_mi(frame_dr[0], frame_dr[1], num_bins, num_genes)

In [ ]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('euclidean')
X = [frame_dr[0], frame_dr[1]]

In [ ]:
%timeit dist.pairwise(X)

In [ ]:
def read_preprocessed_mat(in_file):
    """Read in preprocessed matrix file into a dataframe."""
    if in_file.endswith('.txt'):
        frame = pd.read_csv(in_file, sep="\t", index_col=0).iloc[:, 0:]
    if in_file.endswith('.h5ad') or in_file.endswith('.h5'):
        adata = anndata.read_h5ad(in_file)
        frame = adata.to_df()
    return frame

In [ ]:
adata = anndata.read_h5ad('/Users/lding/Git/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad')

In [ ]:
print(adata.X.shape)

In [ ]:
start = time.time()
indices, dists, forest = sc.neighbors.compute_neighbors_umap(adata.X, n_neighbors=10)
end = time.time()
runtime = end - start
msg = "The runtime for compute_neighbors_umap took {} seconds to complete".format(runtime)
logging.info(msg)

In [ ]:
def calc_norm_mi(arr1, arr2, bins, m):
    """ Calculates a normalized mutual information distance D(X, Y) = 1 - I(X, Y)/H(X, Y) using bin-based method

    It takes gene expression data from single cells, and compares them using standard calculation for
    mutual information and joint entropy. It builds a 2d histogram, which is used to calculate P(arr1, arr2).

    Args:
        arr1 (pandas series): gene expression data for cell 1
        arr2 (pandas series): gene expression data for cell 2
        marginals  (ndarray): marginal probability matrix
        index1         (int): index of cell 1
        index2         (int): index of cell 2
        bins           (int): number of bins
        m              (int): number of genes
    Returns:
        a float between 0 and 1
    """
    fq = fast_histogram.histogram2d(arr1, arr2, range=[[arr1.min(), arr1.max()+1e-9], [arr2.min(), arr2.max()+1e-9]],
                                    bins=(bins, bins)) / float(m)
    sm = np.sum(fq * float(m), axis=1)
    tm = np.sum(fq * float(m), axis=0)
    sm = np.asmatrix(sm / float(sm.sum()))
    tm = np.asmatrix(tm / float(tm.sum()))
    sm_tm = np.matmul(np.transpose(sm), tm)
    div = np.divide(fq, sm_tm, where=sm_tm != 0, out=np.zeros_like(fq))
    ent = np.log(div, where=div != 0, out=np.zeros_like(div))
    agg = np.multiply(fq, ent, out=np.zeros_like(fq), where=fq != 0)
    joint_ent = -np.multiply(fq, np.log(fq, where=fq != 0, out=np.zeros_like(fq)),
                             out=np.zeros_like(fq), where=fq != 0).sum()
    return (joint_ent - agg.sum()) / joint_ent

In [ ]:
num_bins = int((adata.X.shape[0]) ** (1 / 3.0))
num_genes = adata.X.shape[1]
metric_params = {"bins": num_bins, "m": num_genes}

In [ ]:
start = time.time()
indices, dists, forest = sc.neighbors.compute_neighbors_umap(adata.X, n_neighbors=10, metric=calc_norm_mi,
                                                             metric_kwds=metric_params)
end = time.time()
runtime = end - start
msg = "The runtime for compute_neighbors_umap took {} seconds to complete".format(runtime)
logging.info(msg)

In [ ]:
adata = anndata.read_h5ad('/Users/lding/Documents/MICA/Datasets/filtered_gene_bc_matrices/hg19/pbmc33k_preprocessed.h5ad')
frame = adata.to_df()
frame.shape

In [ ]:
frame.to_csv('/Users/lding/Documents/MICA/kgraph/pmbc33k.csv')

In [ ]:
np.arange(1, 12)